In [ ]:
from crossword_solver.crossword_detection_utils import get_grid_from_image, display_image, show_contour_on_image, crop_image_to_crossword_outline
from crossword_solver.square_classification import classify_all_squares, SquareType
from crossword_solver.text_detection import detect_text_for_all_squares
from crossword_solver.solver_helpers import solving_algorithm, display_results, reorder_crossword_hints, find_whole_crossword_candidates, find_suitable_candidates
from crossword_solver.crossword_detection import detect_crossword_from_file
import numpy as np
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from collections import defaultdict
import os
import math
import re
import Levenshtein
import itertools
from tqdm.notebook import trange, tqdm
import time
import pytesseract

In [ ]:
path = '../data/pictures/ristsona_1.png'

In [ ]:
crossword = detect_crossword_from_file(path)

In [ ]:
crossword

In [ ]:
crossword.hints

In [ ]:
reorder_crossword_hints(crossword)

In [ ]:
crossword.hints

In [ ]:
def solving_algorithm(crossword, max_empty_words = 2):
    global timings
    if len(crossword.hints)==0:
        yield crossword
    
    hint = crossword.hints[0]
    start = time.time()
    suitable_candidates = find_suitable_candidates(hint, crossword)
    timings["find suitable candidates"] += time.time()-start

    if len(suitable_candidates)==0 and max_empty_words==0:
        yield crossword
    
    
    if max_empty_words>0:
        new_crossword = deepcopy(crossword)
        new_crossword.hints.pop(0)
        yield from solving_algorithm(new_crossword, max_empty_words-1)
    for candidate in suitable_candidates:
        start = time.time()
        new_crossword = deepcopy(crossword)
        for i, (x, y) in enumerate(hint.coordinates):
            new_crossword.matrix[x, y] = candidate.text[i]
        new_crossword.hints.pop(0)
        new_crossword.score += candidate.weight
        timings["creating new crossword"] += time.time()-start
        yield from solving_algorithm(new_crossword, max_empty_words)
    

In [ ]:
find_whole_crossword_candidates(crossword)

In [ ]:
solutions = solving_algorithm(crossword)

In [ ]:
c = crossword.hints[0].coordinates
matrix = deepcopy(crossword.matrix)
matrix[10:11, 1:13] = np.array(list("kalamarjadsa"))
matrix.T

In [ ]:
timings = defaultdict(int)
results = list(solving_algorithm(crossword, max_empty_words=3))
print(len(results))
print(timings)
display_results(results, topn = 10)

In [ ]:
results = solving_algorithm(crossword)
display_results(results)

In [ ]:
results

In [ ]:
img = cv2.imread(path)
grid = get_grid_from_image(img)
crossword_img = crop_image_to_crossword_outline(img)

In [ ]:
plotting_image = deepcopy(crossword_img)
for gs in grid.flatten():
    plotting_image = cv2.circle(plotting_image, (gs.x_min,gs.y_min), radius=5, color=(0, 255, 0), thickness=-1)
# Display the result
display_image(plotting_image)

In [ ]:
# Get text for all grid_squares
detect_text_for_all_squares(grid)

In [ ]:
classify_all_squares(grid)

In [ ]:
# font 
font = cv2.FONT_HERSHEY_SIMPLEX   
# fontScale 
fontScale = 1   
# Blue color in BGR 
color = (0, 255, 0) 
# Line thickness of 2 px 
thickness = 2
type_to_text = {
    SquareType.HINT : "H",
    SquareType.MULTIHINT : "M",
    SquareType.IRRELEVANT : "I",
    SquareType.EMPTY : "E",
}

for gs in grid.flatten():
    
    t = type_to_text[gs.type]
    plotting_image = cv2.circle(plotting_image, (gs.x_min,gs.y_min), radius=5, color=(0, 255, 0), thickness=-1)
    plotting_image = cv2.putText(plotting_image, t, (gs.x_min+20,gs.y_max-20), font,  fontScale, color, thickness, cv2.LINE_AA)
        
# Display the result
display_image(plotting_image)      


In [ ]:
for gs in grid.flatten():
    if gs.type == SquareType.MULTIHINT:
        print(gs.right_hint_len, gs.bottom_hint_len)
        print(gs.text)

In [ ]:
grid